In [26]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, DataCollatorForTokenClassification
from seqeval.metrics import classification_report
from scipy.stats import ttest_rel
from statsmodels.stats.contingency_tables import mcnemar
import logging

In [43]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [27]:
# Suppress transformers warnings
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [28]:
# Install required packages (if not already installed)
!pip install datasets transformers seqeval pandas numpy scipy statsmodels torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [29]:
# Load the Tamil NER dataset
dataset = load_dataset("xtreme", "PAN-X.ta")
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/919k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/63.3k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [30]:
# Define label list
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

# Define compute_metrics function (assumed from your original code)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    pred_labels = [
        [label_list[p] for p, l in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    report = classification_report(true_labels, pred_labels, output_dict=True)
    return {
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1": report["weighted avg"]["f1-score"],
    }

# Preprocessing functions
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])
        else:
            new_labels.append(labels[word_id] if label_list[labels[word_id]].startswith("I-") else -100)
        previous_word_id = word_id
    return new_labels

def preprocess_data(batch, tokenizer):
    tokenized_inputs = tokenizer(
        batch["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128
    )
    all_word_ids = [tokenized_inputs.word_ids(i) for i in range(len(batch["tokens"]))]
    all_aligned_labels = [
        align_labels_with_tokens(labels, word_ids) for labels, word_ids in zip(batch["ner_tags"], all_word_ids)
    ]
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs

In [31]:
# Load tokenizers and models
tamilbert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-bert")
mbert_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [32]:
# Replace with your Kaggle model paths or Hugging Face Hub paths
tamilbert_model_path = "/kaggle/input/tamilbert-xtreme-5epochs/tamilbert_xtreme_5epochs"  # Update with actual path
mbert_model_path = "/kaggle/input/mbert-xtreme-5epochs/mbert_xtreme_5epochs"  # Update with actual path

tamilbert_model = AutoModelForTokenClassification.from_pretrained(
    tamilbert_model_path, num_labels=len(label_list)
)
mbert_model = AutoModelForTokenClassification.from_pretrained(
    mbert_model_path, num_labels=len(label_list)
)

In [33]:
# Preprocess dataset
train_dataset_tamilbert = train_data.map(lambda batch: preprocess_data(batch, tamilbert_tokenizer), batched=True)
validation_dataset_tamilbert = validation_data.map(lambda batch: preprocess_data(batch, tamilbert_tokenizer), batched=True)
test_dataset_tamilbert = test_data.map(lambda batch: preprocess_data(batch, tamilbert_tokenizer), batched=True)

train_dataset_mbert = train_data.map(lambda batch: preprocess_data(batch, mbert_tokenizer), batched=True)
validation_dataset_mbert = validation_data.map(lambda batch: preprocess_data(batch, mbert_tokenizer), batched=True)
test_dataset_mbert = test_data.map(lambda batch: preprocess_data(batch, mbert_tokenizer), batched=True)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [34]:
# Define data collators
tamilbert_data_collator = DataCollatorForTokenClassification(tamilbert_tokenizer)
mbert_data_collator = DataCollatorForTokenClassification(mbert_tokenizer)

In [36]:
# Set up trainers
tamilbert_trainer = Trainer(
    model=tamilbert_model,
    args=TrainingArguments(
        output_dir="./tamilbert_results",
        eval_strategy="epoch",
        save_strategy="no",
        per_device_eval_batch_size=8,
        report_to="none",
    ),
    eval_dataset=test_dataset_tamilbert,
    tokenizer=tamilbert_tokenizer,
    data_collator=tamilbert_data_collator,
    compute_metrics=compute_metrics,
)

mbert_trainer = Trainer(
    model=mbert_model,
    args=TrainingArguments(
        output_dir="./mbert_results",
        eval_strategy="epoch",
        save_strategy="no",
        per_device_eval_batch_size=8,
        report_to="none",
    ),
    eval_dataset=test_dataset_mbert,
    tokenizer=mbert_tokenizer,
    data_collator=mbert_data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_35/890236073.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  tamilbert_trainer = Trainer(
/tmp/ipykernel_35/890236073.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  mbert_trainer = Trainer(


In [37]:
# Evaluate models
tamilbert_metrics = tamilbert_trainer.evaluate()
mbert_metrics = mbert_trainer.evaluate()

print("\n=== TamilBERT Evaluation Metrics ===")
tamilbert_metrics_df = pd.DataFrame([tamilbert_metrics])
print(tamilbert_metrics_df)

print("\n=== mBERT Evaluation Metrics ===")
mbert_metrics_df = pd.DataFrame([mbert_metrics])
print(mbert_metrics_df)


=== TamilBERT Evaluation Metrics ===
   eval_loss  eval_model_preparation_time  eval_precision  eval_recall  \
0   0.409584                       0.0035        0.843128     0.873489   

    eval_f1  eval_runtime  eval_samples_per_second  eval_steps_per_second  
0  0.857992          4.37                  228.835                 28.604  

=== mBERT Evaluation Metrics ===
   eval_loss  eval_model_preparation_time  eval_precision  eval_recall  \
0   0.417778                       0.0059        0.502293     0.615633   

    eval_f1  eval_runtime  eval_samples_per_second  eval_steps_per_second  
0  0.552242       13.2848                   75.274                  9.409  


In [38]:
# Get token-level predictions
def get_token_predictions(trainer, dataset):
    predictions, labels, _ = trainer.predict(dataset)
    predictions = np.argmax(predictions, axis=-1)
    true_labels = [[l for l in label if l != -100] for label in labels]
    pred_labels = [
        [p for p, l in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    return true_labels, pred_labels

tamilbert_true, tamilbert_pred = get_token_predictions(tamilbert_trainer, test_dataset_tamilbert)
mbert_true, mbert_pred = get_token_predictions(mbert_trainer, test_dataset_mbert)

In [47]:
# McNemar's Test
def perform_mcnemar_test(true_labels, pred_labels_1, pred_labels_2):
    # Flatten predictions and true labels
    flat_true = [label for sent in true_labels for label in sent]
    flat_pred_1 = [label for sent in pred_labels_1 for label in sent]
    flat_pred_2 = [label for sent in pred_labels_2 for label in sent]
    
    # Ensure same length
    min_len = min(len(flat_true), len(flat_pred_1), len(flat_pred_2))
    flat_true = flat_true[:min_len]
    flat_pred_1 = flat_pred_1[:min_len]
    flat_pred_2 = flat_pred_2[:min_len]
    
    # Create contingency table
    both_correct = 0
    model1_correct_model2_incorrect = 0
    model1_incorrect_model2_correct = 0
    both_incorrect = 0
    
    for true, pred1, pred2 in zip(flat_true, flat_pred_1, flat_pred_2):
        if pred1 == true and pred2 == true:
            both_correct += 1
        elif pred1 == true and pred2 != true:
            model1_correct_model2_incorrect += 1
        elif pred1 != true and pred2 == true:
            model1_incorrect_model2_correct += 1
        else:
            both_incorrect += 1
    
    # Contingency table
    table = [[both_correct, model1_correct_model2_incorrect],
             [model1_incorrect_model2_correct, both_incorrect]]
    
    # Perform McNemar's test
    result = mcnemar(table, exact=True)
    print("\n=== McNemar's Test Results ===")
    print(f"Contingency Table:\n{table}")
    print(f"Statistic: {result.statistic}, p-value: {result.pvalue}")
    print(f"Significant difference (p < 0.05): {result.pvalue < 0.05}")



# Run statistical tests
perform_mcnemar_test(tamilbert_true, tamilbert_pred, mbert_pred)



=== McNemar's Test Results ===
Contingency Table:
[[2282, 5415], [94, 314]]
Statistic: 94.0, p-value: 0.0
Significant difference (p < 0.05): True
